<a href="https://colab.research.google.com/github/fothstatshs-design/PROJECTS/blob/main/Automobile_Sales_Dashboard_simulated_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dash pandas plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 43.9 MB/s eta 0:00:00


In [2]:

# =========================================================================
# Automobile Sales Statistics Dashboard
# Optimized to run within Google Colab using modern Dash syntax.
# =========================================================================

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the data
print("Loading data...")
data = pd.read_csv(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
)
print("Data loaded successfully.")

# Initialize the Dash app
app = dash.Dash(__name__)

# Suppress callback exceptions is necessary here because the output-container
# content changes dynamically based on the report type.
app.config.suppress_callback_exceptions = True

# List of years for the dropdown
year_list = [i for i in range(1980, 2024, 1)]

# App layout
app.layout = html.Div([
    # Dashboard title
    html.H1(
        "Automobile Sales Statistics Dashboard",
        style={'textAlign': 'center', 'color': '#503D36', 'font-size': '24px', 'padding': '10px'}
    ),

    # Dropdowns organized in a row
    html.Div([
        # Report Type Dropdown
        html.Div([
            html.Label("Select Report Type:"),
            dcc.Dropdown(
                id='dropdown-statistics',
                options=[
                    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                ],
                placeholder='Select a report type',
                value='Yearly Statistics', # Set a default value
                style={'width': '100%', 'padding': '3px', 'font-size': '20px', 'textAlign': 'center'}
            )
        ], style={'width': '49%', 'display': 'inline-block'}),

        # Year Dropdown
        html.Div([
            html.Label("Select Year:"),
            dcc.Dropdown(
                id='select-year',
                options=[{'label': i, 'value': i} for i in year_list],
                placeholder='Select a year',
                value=2010, # Set a default year
                disabled=False, # Initially enabled
                style={'width': '100%', 'padding': '3px', 'font-size': '20px', 'textAlign': 'center'}
            )
        ], style={'width': '49%', 'display': 'inline-block', 'marginLeft': '10px'})
    ], style={'display': 'flex', 'padding': '0 20px'}),

    html.Br(),

    # Output container where plots will be rendered
    html.Div(id='output-container', className='chart-grid', style={'padding': '10px'})
])

# Callback 1: Enable/disable year dropdown based on report type
@app.callback(
    Output('select-year', 'disabled'),
    Input('dropdown-statistics', 'value')
)
def update_year_dropdown(selected_statistics):
    # Disable the year dropdown if Recession Period Statistics is selected
    if selected_statistics == 'Recession Period Statistics':
        return True
    return False

# Callback 2: Generate plots based on selections
@app.callback(
    Output('output-container', 'children'),
    [Input('dropdown-statistics', 'value'),
     Input('select-year', 'value')]
)
def update_output_container(selected_statistics, input_year):

    if selected_statistics == 'Recession Period Statistics':
        recession_data = data[data['Recession'] == 1]

        # Plot 1: Average Automobile Sales fluctuation over recession period (Line)
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, x='Year', y='Automobile_Sales',
                           title='Average Automobile Sales Fluctuation over Recession Period')
        )

        # Plot 2: Average Vehicles Sold by Vehicle Type (Bar)
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(average_sales, x='Vehicle_Type', y='Automobile_Sales',
                          title='Average Number of Vehicles Sold by Vehicle Type During Recessions')
        )

        # Plot 3: Total Advertisement Expenditure Share by Vehicle Type (Pie)
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec, names='Vehicle_Type', values='Advertising_Expenditure',
                          title='Total Advertisement Expenditure Share by Vehicle Type During Recessions')
        )

        # Plot 4: Effect of Unemployment Rate on Vehicle Type and Sales (Bar)
        # Note: This groups and averages sales by the unemployment rate category
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type',
                          labels={'unemployment_rate':'Unemployment Rate', 'Automobile_Sales':'Average Automobile Sales'},
                          title='Effect of Unemployment Rate on Vehicle Type and Sales During Recessions')
        )

        return [
            # Row 1: R_chart1 and R_chart2
            html.Div([html.Div(R_chart1, style={'width': '50%'}), html.Div(R_chart2, style={'width': '50%'})],
                      style={'display': 'flex'}),
            # Row 2: R_chart3 and R_chart4
            html.Div([html.Div(R_chart3, style={'width': '50%'}), html.Div(R_chart4, style={'width': '50%'})],
                      style={'display': 'flex'})
        ]

    elif selected_statistics == 'Yearly Statistics' and input_year is not None:

        # Filter data for the selected year
        yearly_data = data[data['Year'] == int(input_year)]

        # Plot 1: Yearly Automobile Sales (Line Chart for the WHOLE period)
        # Note: This plot uses the entire dataset 'data' to show context
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas, x='Year', y='Automobile_Sales', title='Yearly Automobile Sales (Whole Period: 1980-2023)')
        )

        # Plot 2: Total Monthly Automobile Sales for the selected year (Line)
        mas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(mas, x='Month', y='Automobile_Sales', title=f'Total Monthly Automobile Sales in {input_year}')
        )

        # Plot 3: Average Vehicles Sold by Vehicle Type in the selected year (Bar)
        avr_vdata = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(avr_vdata, x='Vehicle_Type', y='Automobile_Sales',
                          title=f'Average Vehicles Sold by Vehicle Type in {input_year}')
        )

        # Plot 4: Total Advertisement Expenditure for Each Vehicle in the selected year (Pie)
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(exp_data, names='Vehicle_Type', values='Advertising_Expenditure',
                          title=f'Total Advertisement Expenditure for Each Vehicle in {input_year}')
        )

        return [
            # Row 1: Y_chart1 and Y_chart2
            html.Div([html.Div(Y_chart1, style={'width': '50%'}), html.Div(Y_chart2, style={'width': '50%'})],
                      style={'display': 'flex'}),
            # Row 2: Y_chart3 and Y_chart4
            html.Div([html.Div(Y_chart3, style={'width': '50%'}), html.Div(Y_chart4, style={'width': '50%'})],
                      style={'display': 'flex'})
        ]

    else:
        # Display a prompt if selection is incomplete
        return html.Div([
            html.H3("Please select a Report Type, and a Year (if Yearly Statistics is chosen) to view the analysis.",
                    style={'textAlign': 'center', 'color': '#808080', 'marginTop': '50px'})
        ])

# Run the app
if __name__ == '__main__':
    # Use the modern app.run() syntax for Colab/Jupyter compatibility
    app.run(host='0.0.0.0', port=8050)

Loading data...
Data loaded successfully.


<IPython.core.display.Javascript object>